In [7]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
import numpy as np
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    import os
    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    

In [8]:
matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs\2051\Preferred Network\Preferred_Network_SList_APTG_2051_GGH_MTP_V106A3_CS222_20210305\Daily Demand Matrices"

# peak period - 7 hrs
sov_demand_pk = "TRIPS SOV peak"
hov2_demand_pk = "TRIPS HOV2 peak"
hov3_demand_pk = "TRIPS HOV3 peak"
bike_demand_pk = "TRIPS BIKE peak"
walk_demand_pk = "TRIPS WALK peak"
transit_demand_pk = "TRIPS TOTAL TRANSIT peak"

sov_demand_offpk = "TRIPS SOV offpeak"
hov2_demand_offpk = "TRIPS HOV2 offpeak"
hov3_demand_offpk = "TRIPS HOV3 offpeak"
bike_demand_offpk = "TRIPS BIKE offpeak"
walk_demand_offpk = "TRIPS WALK offpeak"
transit_demand_offpk = "TRIPS TOTAL TRANSIT offpeak"

In [11]:
#matrices
matrix_list = {
    sov_demand_pk : "",
    hov2_demand_pk : "",
    hov3_demand_pk : "",
    bike_demand_pk : "",
    walk_demand_pk : "",
    transit_demand_pk : "",
    sov_demand_offpk : "",
    hov2_demand_offpk : "",
    hov3_demand_offpk : "",
    bike_demand_offpk:"",
    walk_demand_offpk:"",
    transit_demand_offpk:""}

#Import matrices
for matrix_name in matrix_list.keys():
    matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
    print matrix_name
    create_matrix(matrix_id = matrix_list[matrix_name],
                  matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                  matrix_description = matrix_name)

import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",701) 

TRIPS HOV3 offpeak
TRIPS SOV peak
TRIPS TOTAL TRANSIT offpeak
TRIPS BIKE offpeak
TRIPS HOV2 offpeak
TRIPS HOV3 peak
TRIPS SOV offpeak
TRIPS BIKE peak
TRIPS WALK offpeak
TRIPS WALK peak
TRIPS HOV2 peak
TRIPS TOTAL TRANSIT peak
Imported matrix mf107
Imported matrix mf111
Imported matrix mf108
Imported matrix mf114
Imported matrix mf104
Imported matrix mf109
Imported matrix mf110
Imported matrix mf105
Imported matrix mf106
Imported matrix mf115
Imported matrix mf112
Imported matrix mf113
Finished Importing Matrices


In [12]:
def mat_to_pddf(id, scen):
    mat = eb.matrix(id).get_data(scen)
    zones_tupl = mat.indices
    indices = pd.Index(zones_tupl[0])
    indices.name = 'p'
    cols = pd.Index(zones_tupl[1])
    cols.name = 'q'
    matrix = mat.to_numpy()
    return pd.DataFrame(matrix, index=indices, columns=cols)

In [13]:
mf107 = mat_to_pddf("mf107", 701)
mf111 = mat_to_pddf("mf111", 701)
mf108 = mat_to_pddf("mf108", 701)
mf114 = mat_to_pddf("mf114", 701)
mf104 = mat_to_pddf("mf104", 701)
mf109 = mat_to_pddf("mf109", 701)
mf110 = mat_to_pddf("mf110", 701)
mf105 = mat_to_pddf("mf105", 701)
mf106 = mat_to_pddf("mf106", 701)
mf115 = mat_to_pddf("mf115", 701)
mf112 = mat_to_pddf("mf112", 701)
mf113 = mat_to_pddf("mf113", 701)

In [51]:
auto_pass_peak = (mf114 + mf109) - (mf114 / 2.2 + mf109 / 3.3)
auto_pass_offpk = (mf104 + mf108) - (mf108 / 2.2 + mf104 / 3.3)
auto_drive_peak = mf105 + (mf114 / 2.2 + mf109 / 3.3)
auto_drive_offpk = mf110 + (mf108 / 2.2 + mf104 / 3.3)
active_peak = mf111 + mf113
active_offpk = mf107 + mf112
transit_peak = mf115
transit_offpk = mf106

In [52]:
totals = {'auto_drive': [auto_drive_peak.sum().sum(), auto_drive_offpk.sum().sum()],
      'auto_pass':[auto_pass_peak.sum().sum(), auto_pass_offpk.sum().sum()],
      'active':[active_peak.sum().sum(), active_offpk.sum().sum()], 
      'transit':[transit_peak.sum().sum(), transit_offpk.sum().sum()]}

In [53]:
totals_df = pd.DataFrame.from_dict(totals)

In [54]:
totals_fin = totals_df.rename(index = {0 : 'peak', 1 : 'offpk'})
totals_fin.loc['daily',:] = totals_fin.sum(axis=0)
totals_fin.loc[:,'total'] = totals_fin.sum(axis=1)
totals_fin.head(4)

,active,auto_drive,auto_pass,transit,total
peak,1.287652e+06,12224599.0,4837897.0,2629334.25,2.097948e+07
offpk,4.339759e+05,9395972.0,3791637.5,1396536.25,1.501812e+07
daily,1.721628e+06,21620571.0,8629534.5,4025870.50,3.599760e+07


In [56]:
totals_fin.to_csv('C:\Users\pechen\Desktop\GGH\prefnet_trips.csv')